# Subclustering within parent clusters + prognostic validation (no leakage)

This notebook starts from a prepared dataframe `df` that already contains a **parent cluster** column `cluster ∈ {1,2}`.

Goal:
- Find **subclusters** inside each parent cluster (1 and 2) using **only clinical features** (no outcome leakage).
- Run two feature views to avoid self-deception:
  - **View A (clinical phenotype):** numeric + categorical (excludes missingness/flags)
  - **View B (operational/data availability):** numeric + categorical + flags/missing indicators
- Profile subclusters clinically.
- Validate whether subclusters separate prognosis **within each parent cluster** (Kaplan–Meier + log-rank).

Critical anti-leakage rule:
- `event` and `time_days` are **never** used as features for subclustering.
- Outcomes are used **only** in the survival validation section.

Outputs (written to `cluster/`):
- `cluster/df_con_subclusters.csv`
- `cluster/perfil_subclusters.csv`


In [1]:
# Imports & global settings
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import random
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from scipy import stats

from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

RANDOM_STATE = 42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
rng = np.random.default_rng(RANDOM_STATE)

TARGET_EVENT = "event"
TARGET_TIME = "time_days"
PARENT_CLUSTER = "cluster"

OUTPUT_DIR = Path("cluster")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.dpi": 110,
    "axes.grid": True,
    "grid.alpha": 0.25,
})

def _clean_cat_value(x):
    """Normalize category-like values (e.g., '2.0' -> '2') without creating NaNs."""
    if pd.isna(x):
        return x
    s = str(x)
    if s.endswith(".0"):
        head = s[:-2]
        if head.replace("-", "").isdigit():
            return head
    return s

def clean_as_category(s: pd.Series) -> pd.Series:
    return s.map(_clean_cat_value).astype("category")


In [2]:
# 0) Load df (must already include parent cluster column)

CANDIDATE_INPUTS = [
    Path("cluster/df_con_clusters.csv"),
    Path("df_con_clusters.csv"),
    Path("Dades/dt_model.csv"),
]

INPUT_PATH = next((p for p in CANDIDATE_INPUTS if p.exists()), None)
if INPUT_PATH is None:
    raise FileNotFoundError(
        "Could not find an input CSV. Expected one of: " + ", ".join(map(str, CANDIDATE_INPUTS))
    )

df = pd.read_csv(INPUT_PATH)
if "Unnamed: 0" in df.columns and df["Unnamed: 0"].is_unique:
    df = df.set_index("Unnamed: 0")

# Column lists (as specified)
num_cols = [
    "imc",
    "valor_de_ca125",
    "tamano_tumoral",
    "recep_est_porcent",
    "rece_de_Ppor",
    "edad_en_cirugia",
]

cat_cols = [
    "asa",
    "histo_defin",
    "grado_histologi",
    "FIGO2023",
    "afectacion_linf",
    "metasta_distan",
    "AP_centinela_pelvico",
    "AP_ganPelv",
    "AP_glanPaor",
    "beta_cateninap",
]

flag_cols = [
    "histo_defin__from_pre",
    "grado_histologi__from_pre",
    "FIGO2023__from_pre",
    "imc_miss",
    "tamano_tumoral_miss",
    "valor_de_ca125_miss",
    "recep_est_porcent_miss",
    "rece_de_Ppor_miss",
]

required_cols = set(num_cols + cat_cols + flag_cols + [TARGET_EVENT, TARGET_TIME, PARENT_CLUSTER])
missing = sorted(required_cols - set(df.columns))
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Enforce dtypes
for col in num_cols:
    df[col] = df[col].astype("float64")
for col in cat_cols:
    df[col] = clean_as_category(df[col])
for col in flag_cols:
    df[col] = df[col].astype("int8")

df[TARGET_EVENT] = df[TARGET_EVENT].astype("int64")
df[TARGET_TIME] = df[TARGET_TIME].astype("int64")
df[PARENT_CLUSTER] = df[PARENT_CLUSTER].astype("int64")

feature_cols = [c for c in df.columns if c not in [TARGET_EVENT, TARGET_TIME, PARENT_CLUSTER]]
expected_feature_set = set(num_cols + cat_cols + flag_cols)
if set(feature_cols) != expected_feature_set:
    missing_f = sorted(expected_feature_set - set(feature_cols))
    extra_f = sorted(set(feature_cols) - expected_feature_set)
    raise ValueError(
        "feature_cols mismatch vs expected schema. "
        f"Missing: {missing_f}; Extra: {extra_f}"
    )

# Hard checks
if df.isna().any().any():
    na_cols = df.columns[df.isna().any()].tolist()
    raise ValueError(f"NaNs detected in columns: {na_cols}")

event_values = set(df[TARGET_EVENT].unique().tolist())
if not event_values.issubset({0, 1}):
    raise ValueError(f"{TARGET_EVENT} must be binary {{0,1}}; got {sorted(event_values)}")

if (df[TARGET_TIME] < 0).any():
    raise ValueError(f"{TARGET_TIME} contains negative values")

clusters = set(df[PARENT_CLUSTER].unique().tolist())
if clusters != {1, 2}:
    raise ValueError(f"{PARENT_CLUSTER} must have exactly values {{1,2}}; got {sorted(clusters)}")

print("Input path:", INPUT_PATH)
print("df.shape:", df.shape)
print("\n(df.dtypes)\n", df.dtypes)
print("\n(df.head)\n", df.head())


Input path: cluster/df_con_clusters.csv
df.shape: (147, 27)

(df.dtypes)
 imc                           float64
asa                          category
valor_de_ca125                float64
histo_defin                  category
grado_histologi              category
FIGO2023                     category
tamano_tumoral                float64
afectacion_linf              category
metasta_distan               category
AP_centinela_pelvico         category
AP_ganPelv                   category
AP_glanPaor                  category
recep_est_porcent             float64
rece_de_Ppor                  float64
beta_cateninap               category
event                           int64
time_days                       int64
edad_en_cirugia               float64
histo_defin__from_pre            int8
grado_histologi__from_pre        int8
FIGO2023__from_pre               int8
imc_miss                         int8
tamano_tumoral_miss              int8
valor_de_ca125_miss              int8
recep_est_porc

In [3]:
# 1) Mandatory audit

sizes_parent = df[PARENT_CLUSTER].value_counts().sort_index()
print("Parent cluster sizes:")
print(sizes_parent)

print("\nEvent-rate and time_days summary by parent cluster:")
for p in [1, 2]:
    sub = df[df[PARENT_CLUSTER] == p]
    er = float(sub[TARGET_EVENT].mean())
    tmin = int(sub[TARGET_TIME].min())
    tmed = float(sub[TARGET_TIME].median())
    tmax = int(sub[TARGET_TIME].max())
    nevents = int(sub[TARGET_EVENT].sum())
    print(f"- Parent cluster {p}: n={sub.shape[0]}, events={nevents}, event-rate={er:.3f}, time_days min/median/max={tmin}/{tmed:.1f}/{tmax}")


Parent cluster sizes:
cluster
1    86
2    61
Name: count, dtype: int64

Event-rate and time_days summary by parent cluster:
- Parent cluster 1: n=86, events=17, event-rate=0.198, time_days min/median/max=101/1107.0/2303
- Parent cluster 2: n=61, events=20, event-rate=0.328, time_days min/median/max=0/915.0/2481


## 2) Subclustering approach: two views

For each parent cluster (1 and 2) we run two subclustering configurations:

- **View A (clinical phenotype):** `num_cols + cat_cols` (excludes `*_miss` and `__from_pre`)
- **View B (operational/data availability):** `num_cols + cat_cols + flag_cols`

Distance + algorithm:
- Mixed data distance: **Gower** (package if installed, else manual implementation)
- Main algorithm: **hierarchical clustering** with **average linkage** on precomputed distances
- Optional alternative: **k-medoids (PAM)** on precomputed distances (only if available)


In [4]:
# 3) Utilities: Gower distance, clustering, silhouette, bootstrap stability

def gower_distance_matrix_manual(
    X: pd.DataFrame,
    num_cols: list[str],
    cat_cols: list[str],
    flag_cols: list[str],
) -> np.ndarray:
    """Manual Gower distance for mixed data.

    IMPORTANT: numeric ranges are computed within X (here: within the parent cluster).
    """
    n = X.shape[0]
    p = len(num_cols) + len(cat_cols) + len(flag_cols)
    if p == 0:
        raise ValueError("No features provided for Gower distance")

    D = np.zeros((n, n), dtype=np.float64)

    # Numeric part: min-max normalization within the parent cluster
    if num_cols:
        num = X[num_cols].to_numpy(dtype=np.float64)
        mins = num.min(axis=0)
        maxs = num.max(axis=0)
        ranges = maxs - mins
        ranges[ranges == 0.0] = 1.0
        num_norm = (num - mins) / ranges
        for j in range(num_norm.shape[1]):
            col = num_norm[:, j]
            D += np.abs(col[:, None] - col[None, :])

    # Categorical + flags part: 0 if equal else 1
    for col in (cat_cols + flag_cols):
        codes = pd.Categorical(X[col]).codes
        D += (codes[:, None] != codes[None, :]).astype(np.float64)

    D /= float(p)
    np.fill_diagonal(D, 0.0)
    D = (D + D.T) / 2.0
    return D

def compute_gower_distance_matrix(
    X: pd.DataFrame,
    num_cols: list[str],
    cat_cols: list[str],
    flag_cols: list[str],
) -> tuple[np.ndarray, str]:
    """Compute Gower distance matrix (NxN) with a package fallback."""
    try:
        import gower  # type: ignore
        D = np.asarray(gower.gower_matrix(X), dtype=np.float64)
        backend = "gower (package)"
    except ImportError:
        D = gower_distance_matrix_manual(X, num_cols=num_cols, cat_cols=cat_cols, flag_cols=flag_cols)
        backend = "manual"
    return D, backend

def hierarchical_linkage_from_distance(D: np.ndarray, method: str = "average"):
    return linkage(squareform(D, checks=False), method=method)

def hierarchical_cut(Z, K: int) -> np.ndarray:
    return fcluster(Z, t=K, criterion="maxclust").astype(int)

def safe_silhouette_precomputed(D: np.ndarray, labels: np.ndarray) -> float:
    if len(np.unique(labels)) < 2:
        return float("nan")
    counts = pd.Series(labels).value_counts()
    if int(counts.min()) < 2:
        return float("nan")
    try:
        return float(silhouette_score(D, labels, metric="precomputed"))
    except Exception:
        return float("nan")

def silhouette_pca_onehot(
    X: pd.DataFrame,
    labels: np.ndarray,
    num_cols: list[str],
    cat_cols: list[str],
    flag_cols: list[str],
    random_state: int = 42,
) -> float:
    """Approximate silhouette using a PCA embedding of one-hot features.

    Used only if silhouette on precomputed distances is not available.
    """
    if len(np.unique(labels)) < 2:
        return float("nan")
    counts = pd.Series(labels).value_counts()
    if int(counts.min()) < 2:
        return float("nan")

    X_num = StandardScaler().fit_transform(X[num_cols]) if num_cols else np.empty((X.shape[0], 0))
    X_cat = pd.get_dummies(X[cat_cols].astype(str), drop_first=False) if cat_cols else pd.DataFrame(index=X.index)
    X_flag = X[flag_cols].astype(float).to_numpy() if flag_cols else np.empty((X.shape[0], 0))

    X_mat = np.hstack([X_num, X_cat.to_numpy(), X_flag])
    if X_mat.shape[1] < 2 or X_mat.shape[0] < 3:
        return float("nan")

    n_components = int(min(10, X_mat.shape[1], X_mat.shape[0] - 1))
    if n_components < 2:
        return float("nan")

    X_emb = PCA(n_components=n_components, random_state=random_state).fit_transform(X_mat)
    try:
        return float(silhouette_score(X_emb, labels, metric="euclidean"))
    except Exception:
        return float("nan")

def bootstrap_stability_ari(
    D: np.ndarray,
    labels_ref: np.ndarray,
    K: int,
    linkage_method: str,
    B: int,
    rng: np.random.Generator,
) -> tuple[float, float]:
    """Bootstrap stability (ARI) using resampling with replacement.

    We follow an "intersection" approach:
    - draw bootstrap indices with replacement
    - keep unique indices (the intersection set)
    - recluster on the induced distance submatrix
    - compare with the original clustering restricted to the same unique indices
    """
    n = D.shape[0]
    aris = []
    for _ in range(B):
        idx = rng.integers(0, n, size=n)
        uniq = np.unique(idx)
        if uniq.size < max(2, K):
            continue
        D_b = D[np.ix_(uniq, uniq)]
        Z_b = hierarchical_linkage_from_distance(D_b, method=linkage_method)
        labels_b = hierarchical_cut(Z_b, K)
        aris.append(adjusted_rand_score(labels_ref[uniq], labels_b))
    if len(aris) == 0:
        return float("nan"), float("nan")
    return float(np.mean(aris)), float(np.std(aris))

# Optional k-medoids (PAM) if available
try:
    from sklearn_extra.cluster import KMedoids  # type: ignore
    KMEDOIDS_AVAILABLE = True
except ImportError:
    KMEDOIDS_AVAILABLE = False

print("KMedoids available:", KMEDOIDS_AVAILABLE)


KMedoids available: True


In [5]:
# 4) Subclustering within each parent cluster (two views)

LINKAGE_METHOD = "average"
K_CANDIDATES = [2, 3, 4]
B_BOOTSTRAP = 200

view_defs = {
    "A": {
        "label": "View A (clinical phenotype; excludes flags/missingness)",
        "num_cols": num_cols,
        "cat_cols": cat_cols,
        "flag_cols": [],
    },
    "B": {
        "label": "View B (operational/data availability; includes flags/missingness)",
        "num_cols": num_cols,
        "cat_cols": cat_cols,
        "flag_cols": flag_cols,
    },
}

df_sub = df.copy()
df_sub["subcluster_A"] = pd.Series(pd.NA, index=df_sub.index, dtype="Int64")
df_sub["subcluster_B"] = pd.Series(pd.NA, index=df_sub.index, dtype="Int64")
df_sub["subcluster_final"] = pd.Series(0, index=df_sub.index, dtype="Int64")

subclustering_results = {}

def min_cluster_size_threshold(n_parent: int) -> int:
    # As specified: default >=10; if parent is small, relax to >= max(8, 0.15*n_parent)
    if n_parent < 60:
        return int(max(8, np.ceil(0.15 * n_parent)))
    return 10

for parent in [1, 2]:
    df_p = df_sub[df_sub[PARENT_CLUSTER] == parent].copy()
    n_parent = int(df_p.shape[0])
    min_size = min_cluster_size_threshold(n_parent)

    print("\n" + "=" * 90)
    print(f"Parent cluster {parent}: n={n_parent} | min_cluster_size threshold={min_size}")

    subclustering_results[parent] = {"n_parent": n_parent, "min_cluster_size": min_size, "views": {}}

    for view_key, view in view_defs.items():
        cols_view = view["num_cols"] + view["cat_cols"] + view["flag_cols"]
        X_view = df_p[cols_view].copy()

        D, gower_backend = compute_gower_distance_matrix(
            X_view,
            num_cols=view["num_cols"],
            cat_cols=view["cat_cols"],
            flag_cols=view["flag_cols"],
        )
        Z = hierarchical_linkage_from_distance(D, method=LINKAGE_METHOD)

        rows = []
        labels_by_K = {}

        for K in K_CANDIDATES:
            labels_k = hierarchical_cut(Z, K)
            labels_by_K[K] = labels_k

            sizes = pd.Series(labels_k).value_counts().sort_index()
            min_k = int(sizes.min())
            max_k = int(sizes.max())
            status = "ok" if min_k >= min_size else "discarded_min_size"

            sil = safe_silhouette_precomputed(D, labels_k)
            sil_method = "precomputed"
            if np.isnan(sil):
                # Fallback only for reporting (explicitly approximate)
                sil = silhouette_pca_onehot(
                    X_view,
                    labels_k,
                    num_cols=view["num_cols"],
                    cat_cols=view["cat_cols"],
                    flag_cols=view["flag_cols"],
                    random_state=RANDOM_STATE,
                )
                sil_method = "pca_onehot_approx"

            ari_mean, ari_std = bootstrap_stability_ari(
                D,
                labels_ref=labels_k,
                K=K,
                linkage_method=LINKAGE_METHOD,
                B=B_BOOTSTRAP,
                rng=rng,
            )

            rows.append(
                {
                    "parent_cluster": parent,
                    "view": view_key,
                    "K": K,
                    "status": status,
                    "silhouette": float(sil),
                    "silhouette_method": sil_method,
                    "min_cluster_size": min_k,
                    "max_cluster_size": max_k,
                    "balance_ratio_min_over_max": float(min_k / max_k) if max_k > 0 else float("nan"),
                    "cluster_sizes": dict(sizes),
                    "ARI_mean": float(ari_mean),
                    "ARI_std": float(ari_std),
                    "gower_backend": gower_backend,
                    "linkage_method": LINKAGE_METHOD,
                }
            )

        metrics_df = pd.DataFrame(rows).sort_values("K").reset_index(drop=True)
        print("\n---", view["label"], "---")
        print(metrics_df[["K", "status", "min_cluster_size", "cluster_sizes", "silhouette", "silhouette_method", "ARI_mean", "ARI_std"]])

        # Selection rule: prioritize ARI, then balance, then silhouette
        candidates = metrics_df[metrics_df["status"] == "ok"].copy()
        if candidates.empty:
            selected = None
            print(f"No K in {K_CANDIDATES} passes min_cluster_size for parent={parent}, view={view_key}.")
        else:
            candidates = candidates.sort_values(
                ["ARI_mean", "balance_ratio_min_over_max", "silhouette"],
                ascending=[False, False, False],
            )
            selected = candidates.iloc[0].to_dict()
            print(
                f"Selected for parent={parent}, view={view_key}: K={int(selected['K'])} "
                f"(ARI={selected['ARI_mean']:.3f}±{selected['ARI_std']:.3f}, silhouette={selected['silhouette']:.3f}, sizes={selected['cluster_sizes']})"
            )

        # Optional k-medoids comparison for the selected K
        kmedoids_labels = None
        if selected is not None and KMEDOIDS_AVAILABLE:
            K_sel = int(selected["K"])
            km = KMedoids(
                n_clusters=K_sel,
                metric="precomputed",
                method="pam",
                init="heuristic",
                random_state=RANDOM_STATE,
            )
            km.fit(D)
            kmedoids_labels = (km.labels_.astype(int) + 1)  # 1..K
            ari_h_vs_km = adjusted_rand_score(labels_by_K[K_sel], kmedoids_labels)
            print(f"K-medoids available: ARI(hierarchical vs k-medoids)={ari_h_vs_km:.3f}")

        # Store per-view results
        subclustering_results[parent]["views"][view_key] = {
            "view_label": view["label"],
            "cols_view": cols_view,
            "gower_backend": gower_backend,
            "metrics_by_K": metrics_df,
            "selected": selected,
            "labels_by_K": labels_by_K,
            "kmedoids_labels_selected": kmedoids_labels,
        }

        # Assign labels to df_sub (global code = parent*10 + local_label)
        if selected is not None:
            K_sel = int(selected["K"])
            local_labels = labels_by_K[K_sel]
            global_codes = (parent * 10 + local_labels).astype(int)

            target_col = "subcluster_A" if view_key == "A" else "subcluster_B"
            df_sub.loc[df_p.index, target_col] = pd.Series(global_codes, index=df_p.index, dtype="Int64")

# Define recommended subcluster_final for each parent cluster
view_used_final = {}
for parent in [1, 2]:
    idx_parent = df_sub[PARENT_CLUSTER] == parent
    sel_A = subclustering_results[parent]["views"]["A"]["selected"]
    sel_B = subclustering_results[parent]["views"]["B"]["selected"]

    if sel_A is not None:
        df_sub.loc[idx_parent, "subcluster_final"] = df_sub.loc[idx_parent, "subcluster_A"].astype("Int64")
        view_used_final[parent] = "A"
    elif sel_B is not None:
        df_sub.loc[idx_parent, "subcluster_final"] = df_sub.loc[idx_parent, "subcluster_B"].astype("Int64")
        view_used_final[parent] = "B"
    else:
        df_sub.loc[idx_parent, "subcluster_final"] = 0
        view_used_final[parent] = None

print("\nSubcluster counts by parent cluster (using subcluster_final):")
print(df_sub.groupby([PARENT_CLUSTER, "subcluster_final"]).size())



Parent cluster 1: n=86 | min_cluster_size threshold=10

--- View A (clinical phenotype; excludes flags/missingness) ---
   K              status  min_cluster_size              cluster_sizes  \
0  2  discarded_min_size                 1              {1: 85, 2: 1}   
1  3  discarded_min_size                 1        {1: 84, 2: 1, 3: 1}   
2  4  discarded_min_size                 1  {1: 83, 2: 1, 3: 1, 4: 1}   

   silhouette  silhouette_method  ARI_mean   ARI_std  
0         NaN  pca_onehot_approx  0.569870  0.475046  
1         NaN  pca_onehot_approx  0.554537  0.324414  
2         NaN  pca_onehot_approx  0.507781  0.288841  
No K in [2, 3, 4] passes min_cluster_size for parent=1, view=A.


KeyboardInterrupt: 

In [ ]:
# 5) Export df with subclusters

out_path_df = OUTPUT_DIR / "df_con_subclusters.csv"
df_sub.to_csv(out_path_df, index=True)
print("Exported:", out_path_df)


Exported: cluster/df_con_subclusters.csv


## 6) Profiling subclusters (per parent cluster)

For each parent cluster and each `subcluster_final`:
- Numeric: median (IQR)
- Categorical: % per category
- Flags/missing indicators: % of 1s

We also compute differential variables across subclusters:
- Numeric: Kruskal–Wallis
- Categorical/flags: Chi² (or Fisher exact when 2×2 and expected counts are low)


In [ ]:
def fmt_median_iqr(x: pd.Series) -> tuple[float, float, float, str]:
    q1 = float(x.quantile(0.25))
    q3 = float(x.quantile(0.75))
    med = float(x.median())
    return med, q1, q3, f"{med:.2f} [{q1:.2f}, {q3:.2f}]"

def epsilon_squared_from_kruskal(H: float, k: int, n: int) -> float:
    if n <= k:
        return float("nan")
    return max(0.0, float((H - k + 1) / (n - k)))

def cramers_v_from_chi2(chi2: float, n: int, r: int, c: int) -> float:
    denom = n * (min(r - 1, c - 1))
    if denom <= 0:
        return float("nan")
    return float(np.sqrt(chi2 / denom))

profile_rows = []
diff_rows = []
narratives = []

for parent in [1, 2]:
    df_p = df_sub[df_sub[PARENT_CLUSTER] == parent].copy()
    groups = sorted(df_p["subcluster_final"].unique().tolist())

    # Profiling tables
    for sc, sub in df_p.groupby("subcluster_final"):
        n = int(sub.shape[0])

        for col in num_cols:
            med, q1, q3, summary = fmt_median_iqr(sub[col])
            profile_rows.append(
                {
                    "parent_cluster": parent,
                    "subcluster_final": int(sc),
                    "type": "numeric",
                    "variable": col,
                    "level": "",
                    "n": n,
                    "median": med,
                    "q1": q1,
                    "q3": q3,
                    "count": np.nan,
                    "pct": np.nan,
                    "pct1": np.nan,
                    "summary": summary,
                }
            )

        for col in cat_cols:
            vc = sub[col].value_counts(dropna=False)
            for lvl, cnt in vc.items():
                pct = float(cnt / n * 100.0)
                profile_rows.append(
                    {
                        "parent_cluster": parent,
                        "subcluster_final": int(sc),
                        "type": "categorical",
                        "variable": col,
                        "level": str(lvl),
                        "n": n,
                        "median": np.nan,
                        "q1": np.nan,
                        "q3": np.nan,
                        "count": int(cnt),
                        "pct": pct,
                        "pct1": np.nan,
                        "summary": "",
                    }
                )

        for col in flag_cols:
            pct1 = float(sub[col].mean() * 100.0)
            profile_rows.append(
                {
                    "parent_cluster": parent,
                    "subcluster_final": int(sc),
                    "type": "flag",
                    "variable": col,
                    "level": "",
                    "n": n,
                    "median": np.nan,
                    "q1": np.nan,
                    "q3": np.nan,
                    "count": np.nan,
                    "pct": np.nan,
                    "pct1": pct1,
                    "summary": f"%1={pct1:.1f}%",
                }
            )

    # Differential variables (only if at least 2 subclusters)
    if len(groups) >= 2 and not (len(groups) == 1 and groups[0] == 0):
        for col in num_cols:
            group_vals = [df_p.loc[df_p["subcluster_final"] == g, col] for g in groups]
            try:
                H, p = stats.kruskal(*group_vals)
                eff = epsilon_squared_from_kruskal(float(H), k=len(groups), n=int(df_p.shape[0]))
            except Exception:
                H, p, eff = float("nan"), float("nan"), float("nan")
            diff_rows.append(
                {
                    "parent_cluster": parent,
                    "variable": col,
                    "type": "numeric",
                    "test": "kruskal",
                    "stat": float(H),
                    "p_value": float(p),
                    "effect_size": float(eff),
                }
            )

        for col in (cat_cols + flag_cols):
            ct = pd.crosstab(df_p["subcluster_final"], df_p[col])
            try:
                chi2, p_chi2, dof, expected = stats.chi2_contingency(ct, correction=False)
                expected_low = (expected < 5).any()
                test_used = "chi2"
                p_used = float(p_chi2)
                if expected_low and ct.shape == (2, 2):
                    test_used = "fisher_exact"
                    _, p_f = stats.fisher_exact(ct.values)
                    p_used = float(p_f)
                eff = cramers_v_from_chi2(float(chi2), n=int(ct.values.sum()), r=ct.shape[0], c=ct.shape[1])
                stat_val = float(chi2)
            except Exception:
                test_used, p_used, eff, stat_val = "na", float("nan"), float("nan"), float("nan")

            diff_rows.append(
                {
                    "parent_cluster": parent,
                    "variable": col,
                    "type": "categorical" if col in cat_cols else "flag",
                    "test": test_used,
                    "stat": stat_val,
                    "p_value": p_used,
                    "effect_size": float(eff),
                }
            )

    # Narrative profiles (descriptive, no causality)
    overall = df_p
    for sc, sub in df_p.groupby("subcluster_final"):
        candidates = []

        for col in num_cols:
            delta = float(sub[col].median() - overall[col].median())
            candidates.append(
                (
                    abs(delta),
                    f"{col}: median {sub[col].median():.1f} vs parent {overall[col].median():.1f} (Δ={delta:+.1f})",
                )
            )

        for col in flag_cols:
            delta = float(sub[col].mean() - overall[col].mean())
            candidates.append(
                (
                    abs(delta),
                    f"{col}: %1 {sub[col].mean()*100:.1f}% vs parent {overall[col].mean()*100:.1f}% (Δ={delta*100:+.1f}pp)",
                )
            )

        for col in cat_cols:
            top_level = sub[col].value_counts(dropna=False).index[0]
            p_sc = float((sub[col] == top_level).mean())
            p_pa = float((overall[col] == top_level).mean())
            delta = p_sc - p_pa
            candidates.append(
                (
                    abs(delta),
                    f"{col}='{top_level}': {p_sc*100:.1f}% vs parent {p_pa*100:.1f}% (Δ={delta*100:+.1f}pp)",
                )
            )

        candidates.sort(key=lambda x: x[0], reverse=True)
        highlights = [txt for _, txt in candidates[:3]]
        narratives.append(
            {
                "parent_cluster": parent,
                "subcluster_final": int(sc),
                "text": "Perfil descriptivo (no causal): " + "; ".join(highlights),
            }
        )

perfil_subclusters = pd.DataFrame(profile_rows)
perfil_subclusters_path = OUTPUT_DIR / "perfil_subclusters.csv"
perfil_subclusters.to_csv(perfil_subclusters_path, index=False)

print("Exported:", perfil_subclusters_path)
print("\nPerfil_subclusters preview:")
print(perfil_subclusters.head(10))

diff_df = pd.DataFrame(diff_rows)
if not diff_df.empty:
    print("\nTop differential variables by parent cluster:")
    for parent in [1, 2]:
        sub = diff_df[diff_df["parent_cluster"] == parent].sort_values("p_value")
        if sub.empty:
            continue
        print(f"\nParent cluster {parent}:")
        print(sub.head(10))

narr_df = pd.DataFrame(narratives)
print("\nNarrative profiles:")
for parent in [1, 2]:
    sub = narr_df[narr_df["parent_cluster"] == parent].sort_values("subcluster_final")
    print(f"\nParent cluster {parent}:")
    for _, row in sub.iterrows():
        print(f"- subcluster_final={int(row['subcluster_final'])}: {row['text']}")


Exported: cluster/perfil_subclusters.csv

Perfil_subclusters preview:
   parent_cluster  subcluster_final         type           variable    level  \
0               1                 0      numeric                imc            
1               1                 0      numeric     valor_de_ca125            
2               1                 0      numeric     tamano_tumoral            
3               1                 0      numeric  recep_est_porcent            
4               1                 0      numeric       rece_de_Ppor            
5               1                 0      numeric    edad_en_cirugia            
6               1                 0  categorical                asa        1   
7               1                 0  categorical                asa        2   
8               1                 0  categorical                asa        0   
9               1                 0  categorical                asa  Missing   

    n     median         q1         q3  count    

## 7) Prognostic validation within each parent cluster

Within each parent cluster, we compare Kaplan–Meier curves by `subcluster_final` (only if ≥2 subclusters of sufficient size).

We report:
- Event-rate by subcluster
- Global log-rank test (and pairwise with Bonferroni if >2 groups)
- S(1y), S(3y), S(5y) when follow-up allows (365/1095/1825 days)


In [ ]:
# Survival utilities (no external survival libraries)

def kaplan_meier_estimator(time: np.ndarray, event: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Step-wise KM survival curve (timeline, survival), no CI."""
    time = np.asarray(time, dtype=float)
    event = np.asarray(event, dtype=int)

    unique_event_times = np.unique(time[event == 1])
    unique_event_times.sort()

    surv = 1.0
    timeline = [0.0]
    survival = [1.0]

    for t in unique_event_times:
        at_risk = np.sum(time >= t)
        if at_risk == 0:
            continue
        d = np.sum((time == t) & (event == 1))
        surv *= (1.0 - d / at_risk)
        timeline.append(float(t))
        survival.append(float(surv))

    tmax = float(np.max(time))
    if timeline[-1] < tmax:
        timeline.append(tmax)
        survival.append(float(surv))

    return np.asarray(timeline), np.asarray(survival)

def km_survival_at(timeline: np.ndarray, survival: np.ndarray, t0: float) -> float:
    """Return S(t0) from a step KM curve (post-step)."""
    if t0 < 0:
        return float("nan")
    idx = np.searchsorted(timeline, t0, side="right") - 1
    idx = int(np.clip(idx, 0, len(survival) - 1))
    return float(survival[idx])

def logrank_test_multigroup(time: np.ndarray, event: np.ndarray, group: np.ndarray) -> tuple[float, float, int]:
    """Multi-group log-rank test with chi-square approximation."""
    time = np.asarray(time, dtype=float)
    event = np.asarray(event, dtype=int)
    group = np.asarray(group)

    groups = np.unique(group)
    k = len(groups)
    if k < 2:
        return float("nan"), float("nan"), 0

    group_to_idx = {g: i for i, g in enumerate(groups)}
    g_idx = np.array([group_to_idx[g] for g in group], dtype=int)

    event_times = np.unique(time[event == 1])
    event_times.sort()

    O = np.zeros(k, dtype=float)
    E = np.zeros(k, dtype=float)
    V = np.zeros((k, k), dtype=float)

    for t in event_times:
        at_risk = time >= t
        n = int(at_risk.sum())
        if n <= 1:
            continue
        d = int(np.sum((time == t) & (event == 1)))
        if d == 0:
            continue

        n_g = np.zeros(k, dtype=float)
        d_g = np.zeros(k, dtype=float)
        for i in range(k):
            mask = g_idx == i
            n_g[i] = float(np.sum(at_risk & mask))
            d_g[i] = float(np.sum((time == t) & (event == 1) & mask))

        O += d_g
        E += d * (n_g / n)

        factor = d * (n - d) / (n - 1)
        if factor <= 0:
            continue
        p = n_g / n
        V += factor * (np.diag(p) - np.outer(p, p))

    U = O - E
    dof = k - 1
    U_r = U[:-1]
    V_r = V[:-1, :-1]
    stat = float(U_r.T @ np.linalg.pinv(V_r) @ U_r)
    p_value = float(stats.chi2.sf(stat, dof))
    return stat, p_value, dof

survival_results = {}

timepoints = [365, 1095, 1825]

for parent in [1, 2]:
    df_p = df_sub[df_sub[PARENT_CLUSTER] == parent].copy()
    min_size = int(subclustering_results[parent]["min_cluster_size"])
    groups = sorted(df_p["subcluster_final"].unique().tolist())

    # Only consider groups with enough samples
    groups_ok = [g for g in groups if int((df_p["subcluster_final"] == g).sum()) >= min_size]

    print("\n" + "-" * 90)
    print(f"Parent cluster {parent}: view_used_final={view_used_final[parent]} | groups={groups} | groups_ok={groups_ok}")

    if len(groups_ok) < 2:
        print("Not enough subclusters with sufficient size for KM/log-rank within this parent cluster.")
        survival_results[parent] = {"status": "skipped_insufficient_groups"}
        continue

    # Event counts and rates
    rows = []
    for g in groups_ok:
        sub = df_p[df_p["subcluster_final"] == g]
        n = int(sub.shape[0])
        e = int(sub[TARGET_EVENT].sum())
        er = float(sub[TARGET_EVENT].mean())
        rows.append({"subcluster_final": int(g), "n": n, "events": e, "event_rate": er})
    event_table = pd.DataFrame(rows).sort_values("subcluster_final")
    print("\nEvent table:")
    print(event_table)

    total_events = int(df_p[TARGET_EVENT].sum())
    if total_events < 10:
        print("WARNING: Low number of events in this parent cluster -> low power for survival separation.")

    # KM plot
    plt.figure(figsize=(8, 5))
    km_curves = {}
    for g in groups_ok:
        sub = df_p[df_p["subcluster_final"] == g]
        t, s = kaplan_meier_estimator(sub[TARGET_TIME].to_numpy(), sub[TARGET_EVENT].to_numpy())
        km_curves[int(g)] = (t, s)
        plt.step(t, s, where="post", label=f"Subcluster {g} (n={sub.shape[0]})")

    plt.xlabel("Days")
    plt.ylabel("Survival probability")
    plt.title(f"Kaplan–Meier within parent cluster {parent}")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Log-rank
    mask_ok = df_p["subcluster_final"].isin(groups_ok).to_numpy()
    time_ok = df_p.loc[mask_ok, TARGET_TIME].to_numpy()
    event_ok = df_p.loc[mask_ok, TARGET_EVENT].to_numpy()
    group_ok = df_p.loc[mask_ok, "subcluster_final"].to_numpy()

    lr_chi2, lr_p, lr_dof = logrank_test_multigroup(time_ok, event_ok, group_ok)
    print(f"Global log-rank: chi2={lr_chi2:.3f}, dof={lr_dof}, p={lr_p:.4g}")

    pairwise = None
    if len(groups_ok) > 2:
        pairs = list(itertools.combinations(groups_ok, 2))
        m = len(pairs)
        rows = []
        for a, b in pairs:
            mask = df_p["subcluster_final"].isin([a, b]).to_numpy()
            chi2_ab, p_ab, _ = logrank_test_multigroup(
                df_p.loc[mask, TARGET_TIME].to_numpy(),
                df_p.loc[mask, TARGET_EVENT].to_numpy(),
                df_p.loc[mask, "subcluster_final"].to_numpy(),
            )
            rows.append(
                {
                    "a": int(a),
                    "b": int(b),
                    "chi2": float(chi2_ab),
                    "p_value": float(p_ab),
                    "p_bonferroni": float(min(p_ab * m, 1.0)),
                }
            )
        pairwise = pd.DataFrame(rows).sort_values("p_value")
        print("\nPairwise log-rank (Bonferroni):")
        print(pairwise)

    # S(1y), S(3y), S(5y) if follow-up allows
    surv_tp_rows = []
    for g in groups_ok:
        sub = df_p[df_p["subcluster_final"] == g]
        max_fu = float(sub[TARGET_TIME].max())
        t, s = km_curves[int(g)]
        row = {"parent_cluster": parent, "subcluster_final": int(g), "max_followup_days": max_fu}
        for tp in timepoints:
            if max_fu >= tp:
                row[f"S_{tp}d"] = km_survival_at(t, s, tp)
            else:
                row[f"S_{tp}d"] = float("nan")
        surv_tp_rows.append(row)

    surv_tp = pd.DataFrame(surv_tp_rows)
    print("\nSurvival probabilities at fixed timepoints (NaN if insufficient follow-up):")
    print(surv_tp)

    survival_results[parent] = {
        "status": "ok",
        "groups_ok": groups_ok,
        "event_table": event_table,
        "logrank_global": {"chi2": float(lr_chi2), "p_value": float(lr_p), "dof": int(lr_dof)},
        "logrank_pairwise": pairwise,
        "survival_timepoints": surv_tp,
    }



------------------------------------------------------------------------------------------
Parent cluster 1: view_used_final=None | groups=[0] | groups_ok=[0]
Not enough subclusters with sufficient size for KM/log-rank within this parent cluster.

------------------------------------------------------------------------------------------
Parent cluster 2: view_used_final=None | groups=[0] | groups_ok=[0]
Not enough subclusters with sufficient size for KM/log-rank within this parent cluster.


## 8) Interpretation and final decision

We summarize, for each parent cluster:
- Whether View A has robust substructure
- Whether View B has robust substructure
- Whether there is survival separation
- Which variables define the subgroups

Conclusion rule-of-thumb:
- If View A is robust → report as **clinical subphenotypes**.
- If only View B is robust and top drivers are missingness/flags → report as **data availability profiles**.


In [ ]:
def top_differential_variables(diff_df: pd.DataFrame, parent: int, top_n: int = 5) -> pd.DataFrame:
    if diff_df.empty:
        return pd.DataFrame()
    sub = diff_df[diff_df["parent_cluster"] == parent].sort_values("p_value")
    return sub.head(top_n)

summary_rows = []
for parent in [1, 2]:
    sel_A = subclustering_results[parent]["views"]["A"]["selected"]
    sel_B = subclustering_results[parent]["views"]["B"]["selected"]
    used = view_used_final[parent]

    top_vars = top_differential_variables(diff_df, parent=parent, top_n=8)
    top_vars_list = top_vars["variable"].tolist() if not top_vars.empty else []
    n_flags_in_top = sum(v in flag_cols for v in top_vars_list)
    flag_fraction = (n_flags_in_top / len(top_vars_list)) if top_vars_list else 0.0

    surv = survival_results.get(parent, {})
    lr_p = float(surv.get("logrank_global", {}).get("p_value", np.nan))

    interpretation = ""
    if used == "A":
        interpretation = "Clinical subphenotypes (View A selected)."
    elif used == "B":
        if flag_fraction >= 0.5:
            interpretation = "Likely data availability / missingness-driven (View B only; many top drivers are flags)."
        else:
            interpretation = "Operational profile (View B selected), but drivers are not dominated by flags."
    else:
        interpretation = "No robust substructure detected (subcluster_final=0)."

    summary_rows.append(
        {
            "parent_cluster": parent,
            "n_parent": subclustering_results[parent]["n_parent"],
            "view_A_selected": sel_A is not None,
            "view_A_K": int(sel_A["K"]) if sel_A else np.nan,
            "view_B_selected": sel_B is not None,
            "view_B_K": int(sel_B["K"]) if sel_B else np.nan,
            "view_used_final": used if used else "none",
            "logrank_p_within_parent": lr_p,
            "top_drivers": ", ".join(top_vars_list[:5]),
            "flag_fraction_in_top8": flag_fraction,
            "interpretation": interpretation,
        }
    )

summary_df = pd.DataFrame(summary_rows)
print("Final summary by parent cluster:")
print(summary_df)

# Collect results object
results = {
    "RANDOM_STATE": RANDOM_STATE,
    "input_path": str(INPUT_PATH),
    "output_dir": str(OUTPUT_DIR),
    "subclustering": subclustering_results,
    "view_used_final": view_used_final,
    "df_sub": df_sub,
    "perfil_subclusters": perfil_subclusters,
    "diff_subclusters": diff_df,
    "narratives": narr_df,
    "survival_results": survival_results,
    "summary": summary_df,
    "exports": {
        "df_con_subclusters_csv": str(OUTPUT_DIR / "df_con_subclusters.csv"),
        "perfil_subclusters_csv": str(OUTPUT_DIR / "perfil_subclusters.csv"),
    },
}

print("\nExports:")
for k, v in results["exports"].items():
    print(f"- {k}: {v}")


Final summary by parent cluster:
   parent_cluster  n_parent  view_A_selected  view_A_K  view_B_selected  \
0               1        86            False       NaN            False   
1               2        61            False       NaN            False   

   view_B_K view_used_final  logrank_p_within_parent top_drivers  \
0       NaN            none                      NaN               
1       NaN            none                      NaN               

   flag_fraction_in_top8                                     interpretation  
0                    0.0  No robust substructure detected (subcluster_fi...  
1                    0.0  No robust substructure detected (subcluster_fi...  

Exports:
- df_con_subclusters_csv: cluster/df_con_subclusters.csv
- perfil_subclusters_csv: cluster/perfil_subclusters.csv
